# Testing for Cointegration and Stationarity

In [743]:
# API to be used
import yfinance as yf
# principal data manipulation tools
import numpy as np 
import pandas as pd
import matplotlib as plt

# data visualization
import seaborn as sns

# math
import statsmodels.api as sm
from statsmodels.regression.linear_model import OLS # least squares
from statsmodels.tsa.stattools import adfuller # augmented dicky fuller test
from statsmodels.tsa.stattools import coint # cointegration (not corrrelation)
from sklearn.model_selection import train_test_split
import datetime

# GUI
from ipywidgets import widgets, interact
from IPython.display import display

# local imports
# %run dictionary.ipynb

### Add securities below one at a time

In [379]:
ticker_list = []

add_ticker = widgets.Text(
    value = '',
    placeholder = 'Type a valid Ticker',
    description = 'Ticker name:',
)

display(add_ticker)


def callback(widget):
    ticker_list.append(widget.value) 
    
add_ticker.on_submit(callback)

# TODO: functionality for multiple 
def multiple_securities():
    return

Text(value='', description='Ticker name:', placeholder='Type a valid Ticker')

In [744]:
# sample list
print(ticker_list)
print()
print(len(ticker_list))

['AAPL', 'MSFT', 'FB', 'BOX', 'U', 'CRM', 'TWTR', 'SNAP', 'AMD', 'SQ', 'ABNB', 'PLTR', 'TSLA', 'AMZN', 'ROKU', 'NVDA', 'GOOG', 'NFLX', 'UBER', 'LYFT']

20


In [745]:
# scratch
ticker = yf.Ticker(ticker_list[i])
hist = ticker.history(start = '2020-01-01', end = today)
hist = hist['Close']
hist[0]

43.58000183105469

### DataFrame set up

In [746]:
my_dict = {}

# create a dictionary with all of the tickers as keys and all of the values in the form [Date, Close]
for i in range(len(ticker_list)):
    ticker = yf.Ticker(ticker_list[i])
#     about two years of data
    hist = ticker.history(start = '2020-01-01', end = today)
#     iterate over the hist data
    for j in range(len(hist)):
        my_dict[ticker_list[i]] = hist['Close']

    


In [747]:
df = pd.DataFrame(my_dict, index = hist['Close'].index)
# populate each column
# for i in range(len(hist['Close'])):
#     df[hist['Close'].index[i]][ticker_list[i]] = hist['Close'][i]

df


# df.from_dict(my_dict)
# s = pd.Series
# s.to_dict(hist['Close'])

,AAPL,MSFT,FB,BOX,U,CRM,TWTR,SNAP,AMD,SQ,ABNB,PLTR,TSLA,AMZN,ROKU,NVDA,GOOG,NFLX,UBER,LYFT
Date,,,,,,,,,,,,,,,,,,,,
2020-01-02,73.988480,157.615112,209.779999,17.240000,NaN,166.990005,32.299999,16.780001,49.099998,63.830002,NaN,NaN,86.052002,1898.010010,137.100006,59.826447,1367.369995,329.809998,30.990000,43.580002
2020-01-03,73.269150,155.652512,208.669998,16.940001,NaN,166.169998,31.520000,16.750000,48.599998,63.000000,NaN,NaN,88.601997,1874.969971,137.509995,58.868862,1360.660034,325.899994,31.370001,43.189999
2020-01-06,73.852982,156.054855,212.600006,17.320000,NaN,173.449997,31.639999,16.780001,48.389999,62.570000,NaN,NaN,90.307999,1902.880005,143.369995,59.115734,1394.209961,335.829987,31.580000,43.099998
2020-01-07,73.505646,154.631989,213.059998,16.969999,NaN,176.000000,32.540001,17.030001,48.250000,64.589996,NaN,NaN,93.811996,1906.859985,138.190002,59.831425,1393.339966,330.750000,32.810001,44.250000
2020-01-08,74.688080,157.095032,215.220001,16.860001,NaN,177.330002,33.049999,16.670000,47.830002,67.599998,NaN,NaN,98.428001,1891.969971,134.639999,59.943645,1404.319946,339.260010,33.930000,45.080002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-16,172.259995,324.899994,334.899994,25.629999,133.740005,253.119995,43.130001,44.700001,138.639999,165.880005,156.380005,18.340000,926.919983,3377.419922,219.009995,283.869995,2896.770020,591.059998,37.700001,37.459999
2021-12-17,171.139999,323.799988,333.790009,26.219999,138.410004,252.929993,43.070000,45.290001,137.750000,167.059998,157.910004,19.059999,932.570007,3400.350098,236.460007,278.010010,2856.060059,586.729980,39.680000,38.070000
2021-12-20,169.750000,319.910004,325.450012,25.930000,137.289993,247.210007,42.980000,44.419998,135.800003,158.300003,157.229996,17.959999,899.940002,3341.580078,228.059998,277.190002,2848.030029,593.739990,39.700001,38.619999


In [739]:
# Everything from here below is from https://towardsdatascience.com/bring-your-jupyter-notebook-to-life-with-interactive-widgets-bc12e03f0916 

In [742]:
# using widgets

widgets.IntSlider(
    min = 0,
    max = len(ticker_dict) - 1,
    step = 1,
    description = 'Slider:',
    value = 0
)

slider = widgets.IntSlider()
display(slider)

IntSlider(value=0)

In [741]:
# linking two widgets
slider = widgets.IntSlider()
text = widgets.IntText()
display(slider, text)
widgets.jslink((slider, 'value'), (text, 'value'))

IntSlider(value=0)

IntText(value=0)

Link(source=(IntSlider(value=0), 'value'), target=(IntText(value=0), 'value'))

In [141]:
print(dir(widgets))

['Accordion', 'AppLayout', 'Audio', 'BoundedFloatText', 'BoundedIntText', 'Box', 'Button', 'ButtonStyle', 'CallbackDispatcher', 'Checkbox', 'Color', 'ColorPicker', 'Combobox', 'Controller', 'CoreWidget', 'DOMWidget', 'DatePicker', 'Datetime', 'Dropdown', 'FileUpload', 'FloatLogSlider', 'FloatProgress', 'FloatRangeSlider', 'FloatSlider', 'FloatText', 'GridBox', 'GridspecLayout', 'HBox', 'HTML', 'HTMLMath', 'Image', 'IntProgress', 'IntRangeSlider', 'IntSlider', 'IntText', 'Label', 'Layout', 'NumberFormat', 'Output', 'Password', 'Play', 'RadioButtons', 'Select', 'SelectMultiple', 'SelectionRangeSlider', 'SelectionSlider', 'SliderStyle', 'Style', 'Tab', 'Text', 'Textarea', 'ToggleButton', 'ToggleButtons', 'ToggleButtonsStyle', 'TwoByTwoLayout', 'VBox', 'Valid', 'ValueWidget', 'Video', 'Widget', '__builtins__', '__cached__', '__doc__', '__file__', '__jupyter_widgets_base_version__', '__jupyter_widgets_controls_version__', '__loader__', '__name__', '__package__', '__path__', '__protocol_vers

In [740]:
# widget events
btn = widgets.Button(description='Medium')
display(btn)
def btn_eventhandler(obj):
    print(ticker_dict[0].format(obj.description))
btn.on_click(btn_eventhandler)

Button(description='Medium', style=ButtonStyle())

AAPL
